In [10]:
# importing required modules
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import fashion_mnist

In [11]:
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    # Legacy Python that doesn't verify HTTPS certificates by default
    pass
else:
    # Handle target environment that doesn't support HTTPS verification
    ssl._create_default_https_context = _create_unverified_https_context

In [12]:
# loading data
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [13]:
# checking if all data is loaded properly
assert x_train.shape == (60000, 28, 28)
assert x_test.shape == (10000, 28, 28)
assert y_train.shape == (60000,)
assert y_test.shape == (10000,)

In [14]:
# flattening and normalizing the input data
x_train = x_train.reshape(x_train.shape[0], -1) / 255.0
x_test = x_test.reshape(x_test.shape[0], -1) / 255.0

In [15]:
# one-hot encoding (creating an identity matrix corresponding to the number of categories)
def one_hot_encode(y, num_classes):
    return np.eye(num_classes)[y]

In [16]:
# applying one-hot encoding (creating 10x10 matrix for the 10 categories of digits)
y_train_encoded = one_hot_encode(y_train, 10)
y_test_encoded = one_hot_encode(y_test, 10)

In [17]:
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size, learning_rate=0.01, l1_lambda=0.0, l2_lambda=0.0):
        self.W1 = np.random.randn(input_size, hidden_size) * 0.01
        self.b1 = np.zeros((1, hidden_size))
        self.W2 = np.random.randn(hidden_size, output_size) * 0.01
        self.b2 = np.zeros((1, output_size))
        self.learning_rate = learning_rate
        self.l1_lambda = l1_lambda
        self.l2_lambda = l2_lambda

    def softmax(self, x):
        exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
        return exp_x / np.sum(exp_x, axis=1, keepdims=True)

    def forward(self, X):
        self.z1 = np.dot(X, self.W1) + self.b1
        self.a1 = np.tanh(self.z1)
        self.z2 = np.dot(self.a1, self.W2) + self.b2
        self.a2 = self.softmax(self.z2)
        return self.a2

    def backward(self, X, y, output):
        m = X.shape[0]
        output_error = output - y
        hidden_error = np.dot(output_error, self.W2.T) * (1 - np.tanh(self.z1) ** 2)

        dW2 = np.dot(self.a1.T, output_error) / m
        db2 = np.sum(output_error, axis=0, keepdims=True) / m
        dW1 = np.dot(X.T, hidden_error) / m
        db1 = np.sum(hidden_error, axis=0, keepdims=True) / m

        # L1 regularization
        dW1 += self.l1_lambda * np.sign(self.W1)
        dW2 += self.l1_lambda * np.sign(self.W2)

        # L2 regularization
        dW1 += self.l2_lambda * self.W1
        dW2 += self.l2_lambda * self.W2

        self.W1 -= self.learning_rate * dW1
        self.b1 -= self.learning_rate * db1
        self.W2 -= self.learning_rate * dW2
        self.b2 -= self.learning_rate * db2

    def train(self, X, y, epochs):
        for epoch in range(epochs):
            output = self.forward(X)
            self.backward(X, y, output)
            if epoch % 10 == 0:
                loss = -np.mean(np.sum(y * np.log(output + 1e-10), axis=1))
                print(f"Epoch {epoch}, Loss: {loss:.4f}")

    def predict(self, X):
        output = self.forward(X)
        return np.argmax(output, axis=1)

In [18]:
# Hyperparameters
input_size = 784  # 28x28 pixels
hidden_size = 64  # Number of neurons in the hidden layer
output_size = 10   # 10 classes for digits 0-9
learning_rate = 0.5
epochs = 1000

In [ ]:
# Train without regularization
nn_no_reg = NeuralNetwork(input_size, hidden_size, output_size, learning_rate)
nn_no_reg.train(x_train, y_train_encoded, epochs)
predictions_no_reg = nn_no_reg.predict(x_test)
accuracy_no_reg = np.mean(predictions_no_reg == y_test)
print(f"Test Accuracy (No Regularization): {accuracy_no_reg:.4f}")

Epoch 0, Loss: 2.3017
Epoch 10, Loss: 1.4750
Epoch 20, Loss: 1.1990
Epoch 30, Loss: 0.9772
Epoch 40, Loss: 0.7969
Epoch 50, Loss: 0.7436
Epoch 60, Loss: 0.6947
Epoch 70, Loss: 0.6626
Epoch 80, Loss: 0.6397
Epoch 90, Loss: 0.6432
Epoch 100, Loss: 0.6085
Epoch 110, Loss: 0.5928
Epoch 120, Loss: 0.5678
Epoch 130, Loss: 0.5520
Epoch 140, Loss: 0.5371
Epoch 150, Loss: 0.5466
Epoch 160, Loss: 0.5311
Epoch 170, Loss: 0.5232
Epoch 180, Loss: 0.5151
Epoch 190, Loss: 0.4932
Epoch 200, Loss: 0.5121
Epoch 210, Loss: 0.4896
Epoch 220, Loss: 0.6267
